## 마을공동시설(경로당) 데이터

In [1]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
senior_center = pd.read_csv(
    filepath_or_buffer = "../data/경로당.csv", 
    encoding = "cp949"
    )
senior_center["주소"] = "경상남도" + " " + senior_center["시군명"] + " " + senior_center["주소"]
senior_center

,연번,시군명,시설명,주소
0,1,창원시,가곡 경로당,경상남도 창원시 의창구 동읍 신촌본포로442번길 26
1,2,창원시,고양 경로당,경상남도 창원시 의창구 동읍 백월로 540
2,3,창원시,곡목 경로당,경상남도 창원시 의창구 동읍 동읍로 359번길 43
3,4,창원시,금동 경로당,경상남도 창원시 의창구 동읍 동읍로 835번길 7
4,5,창원시,금산 경로당,경상남도 창원시 의창구 동읍 동읍로 670번길 6
...,...,...,...,...
7529,7530,합천군,(용주)성산경로당,경상남도 합천군 용주면 성산3길 31
7530,7531,합천군,(용주)성산2구경로당,경상남도 합천군 용주면 성산길 18
7531,7532,합천군,(용주)손목경로당,경상남도 합천군 용주면 손목2길 13
7532,7533,합천군,(용주)손목1구할머니경로당,경상남도 합천군 용주면 황계폭포로 1395


In [4]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in senior_center.iterrows():
    address = row['주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({ "경도": x, "위도": y})
        else:
            coordinates.append({"경도": None, "위도": None})
    else:
        coordinates.append({"경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)

# 결과 출력
coord_df

,경도,위도
0,128.644385896,35.359032004
1,128.654864588,35.305038793
2,128.661179593,35.298651114
3,128.666221058,35.333232725
4,128.664271216,35.323145187
...,...,...
7529,128.135081348,35.556141195
7530,128.148858647,35.552522829
7531,128.127954252,35.553945293
7532,128.130041340,35.552212500


In [6]:
# 데이터프레임 합치기
senior_center = pd.concat([senior_center, coord_df], axis=1)
senior_center = senior_center[['시설명','주소','경도','위도']]
senior_center

,시설명,주소,경도,위도
0,가곡 경로당,경상남도 창원시 의창구 동읍 신촌본포로442번길 26,128.644385896,35.359032004
1,고양 경로당,경상남도 창원시 의창구 동읍 백월로 540,128.654864588,35.305038793
2,곡목 경로당,경상남도 창원시 의창구 동읍 동읍로 359번길 43,128.661179593,35.298651114
3,금동 경로당,경상남도 창원시 의창구 동읍 동읍로 835번길 7,128.666221058,35.333232725
4,금산 경로당,경상남도 창원시 의창구 동읍 동읍로 670번길 6,128.664271216,35.323145187
...,...,...,...,...
7529,(용주)성산경로당,경상남도 합천군 용주면 성산3길 31,128.135081348,35.556141195
7530,(용주)성산2구경로당,경상남도 합천군 용주면 성산길 18,128.148858647,35.552522829
7531,(용주)손목경로당,경상남도 합천군 용주면 손목2길 13,128.127954252,35.553945293
7532,(용주)손목1구할머니경로당,경상남도 합천군 용주면 황계폭포로 1395,128.130041340,35.552212500


## 목욕장 데이터

In [20]:
bathhouse = pd.read_csv(
    filepath_or_buffer = "../data/목욕탕 데이터.csv", 
    encoding = "cp949"
    )  

bathhouse = bathhouse[['도로명전체주소','사업장명']].dropna()
bathhouse = bathhouse[bathhouse["도로명전체주소"].str.contains("경상남도")].reset_index(drop=True)
bathhouse.rename(columns={'도로명전체주소':'주소', '사업장명':'시설명'}, inplace=True)

<bound method NDFrame.head of                                           주소           시설명
0                 경상남도 김해시 금관대로1277번길 5 (내동)         내동소정탕
1               경상남도 진주시 대신로457번길 12-3 (초전동)       우리사우나헬스
2                   경상남도 진주시 석갑로53번길 2 (평거동)         백옥목욕탕
3                  경상남도 진주시 평거로13번길 27 (평거동)           억수탕
4                경상남도 진주시 촉석로225번길 8-1 (수정동)           대신탕
...                                      ...           ...
1108                 경상남도 남해군 이동면 남서대로 208-6       성남작은목욕탕
1109                  경상남도 하동군 금성면 경제산업로 509       섬진강문화센터
1110                   경상남도 하동군 하동읍 경서대로 162         명성사우나
1111  경상남도 하동군 악양면 악양동로 366-13, 악양면종합복지회관 1층  악양면 복지회관 목욕탕
1112                    경상남도 하동군 양보면 진양로 726       양보복지목욕탕

[1113 rows x 2 columns]>

In [21]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in bathhouse.iterrows():
    address = row['주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({"경도": x, "위도": y})
        else:
            coordinates.append({"경도": None, "위도": None})
    else:
        coordinates.append({"경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)

# 결과 출력
coord_df

,경도,위도
0,128.859235587,35.239196059
1,128.114498050,35.201710463
2,128.064658441,35.176875090
3,128.062314631,35.174167406
4,128.086277181,35.197065859
...,...,...
1108,127.939973982,34.779897288
1109,None,None
1110,127.751084639,35.069331241
1111,127.709871235,35.162318104


In [22]:
bathhouse = pd.concat([bathhouse, coord_df], axis=1)
bathhouse

,주소,시설명,경도,위도
0,경상남도 김해시 금관대로1277번길 5 (내동),내동소정탕,128.859235587,35.239196059
1,경상남도 진주시 대신로457번길 12-3 (초전동),우리사우나헬스,128.114498050,35.201710463
2,경상남도 진주시 석갑로53번길 2 (평거동),백옥목욕탕,128.064658441,35.176875090
3,경상남도 진주시 평거로13번길 27 (평거동),억수탕,128.062314631,35.174167406
4,경상남도 진주시 촉석로225번길 8-1 (수정동),대신탕,128.086277181,35.197065859
...,...,...,...,...
1108,경상남도 남해군 이동면 남서대로 208-6,성남작은목욕탕,127.939973982,34.779897288
1109,경상남도 하동군 금성면 경제산업로 509,섬진강문화센터,None,None
1110,경상남도 하동군 하동읍 경서대로 162,명성사우나,127.751084639,35.069331241
1111,"경상남도 하동군 악양면 악양동로 366-13, 악양면종합복지회관 1층",악양면 복지회관 목욕탕,127.709871235,35.162318104


In [23]:
convenience_facility = pd.concat([senior_center, bathhouse], axis=0).reset_index(drop=[True])
convenience_facility.head

<bound method NDFrame.head of                시설명                                      주소             경도  \
0           가곡 경로당           경상남도 창원시 의창구 동읍 신촌본포로442번길 26  128.644385896   
1          고양 경로당                 경상남도 창원시 의창구 동읍 백월로 540   128.654864588   
2          곡목 경로당            경상남도 창원시 의창구 동읍 동읍로 359번길 43   128.661179593   
3          금동 경로당             경상남도 창원시 의창구 동읍 동읍로 835번길 7   128.666221058   
4          금산 경로당             경상남도 창원시 의창구 동읍 동읍로 670번길 6   128.664271216   
...            ...                                     ...            ...   
8642       성남작은목욕탕                 경상남도 남해군 이동면 남서대로 208-6  127.939973982   
8643       섬진강문화센터                  경상남도 하동군 금성면 경제산업로 509           None   
8644         명성사우나                   경상남도 하동군 하동읍 경서대로 162  127.751084639   
8645  악양면 복지회관 목욕탕  경상남도 하동군 악양면 악양동로 366-13, 악양면종합복지회관 1층  127.709871235   
8646       양보복지목욕탕                    경상남도 하동군 양보면 진양로 726  127.845889550   

                위도  
0     35.359032004  
1  

In [24]:
convenience_facility.to_csv("../data/생활편익시설.csv", index=False)